In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam


In [10]:


# Data augmentation para entrenamiento
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,       # Rotar la imagen aleatoriamente entre 0 y 20 grados
    width_shift_range=0.2,   # Desplazamiento horizontal de la imagen
    height_shift_range=0.2,  # Desplazamiento vertical de la imagen
    shear_range=0.2,         # Aplicar una transformación de corte
    zoom_range=0.2,          # Aplicar zoom aleatorio
    horizontal_flip=True,    # Voltear horizontalmente
    fill_mode='nearest')     # Rellenar los píxeles vacíos

# Para el conjunto de validación solo normalizamos (sin aumento de datos)
val_datagen = ImageDataGenerator(rescale=1./255)

# Cargar datos desde las carpetas
train_generator = train_datagen.flow_from_directory(
    'TIRADS/',  # Carpeta de entrenamiento
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
    'test_data/',  # Carpeta de validación
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')


Found 53 images belonging to 5 classes.
Found 11 images belonging to 5 classes.


In [11]:
# Crear el modelo con regularización y Dropout
model = Sequential()

# Primera capa Conv2D con regularización L2
model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.001), input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Segunda capa Conv2D con regularización L2
model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.001)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Tercera capa Conv2D con regularización L2
model.add(Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.001)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Aplanar y agregar Dropout para regularización adicional
model.add(Flatten())
model.add(Dropout(0.5))  # Apagar el 50% de las neuronas

# Capa totalmente conectada (Densa)
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.5))  # Otro Dropout

# Capa de salida con softmax para clasificación en múltiples categorías (TIRADS 1-5)
model.add(Dense(5, activation='softmax'))

# Compilar el modelo con el optimizador Adam
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [13]:
# Entrenar el modelo
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=100,  # Ajusta el número de épocas
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size)


ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.

In [25]:
# Evaluar el modelo en el conjunto de validación
val_loss, val_acc = model.evaluate(validation_generator)
print(f'Loss en validación: {val_loss}')
print(f'Accuracy en validación: {val_acc}')


ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.

In [15]:
# Guardar el modelo en formato HDF5
model.save('modelo_tirads_regularizado.h5')

# Convertir a TensorFlow.js para su uso en la web
#!tensorflowjs_converter --input_format keras modelo_tirads_regularizado.h5 carpeta_web_model


In [ ]:
# Para nuevas imágenes de prueba
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    'new_test_images',  # Carpeta con nuevas imágenes
    target_size=(224, 224),
    batch_size=1,  # Procesar una imagen a la vez
    class_mode='categorical',
    shuffle=False)

# Hacer predicciones
predictions = model.predict(test_generator)


In [ ]:
# !pip install tensorflowjs


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 80.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
      Successfully uninstalled packaging-24.1


In [ ]:
# Exportar el modelo entrenado a formato de TensorFlow.js
import tensorflowjs as tfjs

# Directorio donde guardar el modelo convertido
tfjs_target_dir = '/content/drive/MyDrive/models/somia_v2/'

# Convertir el modelo a TensorFlow.js
tfjs.converters.save_keras_model(model, tfjs_target_dir)


failed to lookup keras version from the file,
    this is likely a weight only file


In [1]:
# Cargar librerías necesarias
from tensorflow.keras.preprocessing import image
import numpy as np
from PIL import Image

# Ruta de la imagen
img_path = 'TIRADS/TIRADS_2/0.png'

# Cargar la imagen y convertirla a RGB (esto eliminará el canal alfa si está presente)
img = Image.open(img_path).convert('RGB')

# Redimensionar la imagen a las dimensiones que tu modelo espera (224x224, por ejemplo)
img = img.resize((224, 224))

# Convertir la imagen a un array numpy y normalizar (0 a 1)
img_array = np.array(img) / 255.0

# Expandir dimensiones para representar el batch (tu modelo probablemente espera una dimensión extra)
img_array = np.expand_dims(img_array, axis=0)

# Realizar la predicción usando tu modelo
predictions = model.predict(img_array)

# Asumimos que tu modelo tiene 5 clases (TIRADS 1 a TIRADS 5)
predicted_class = np.argmax(predictions, axis=1)

# Lista de etiquetas para las predicciones
class_labels = ['TIRADS 1', 'TIRADS 2', 'TIRADS 3', 'TIRADS 4', 'TIRADS 5']

# Mostrar el resultado de la predicción
print(f"La imagen se clasifica como: {class_labels[predicted_class[0]]}")


NameError: name 'model' is not defined